In [ ]:
import sys
import os
import argparse

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [15]:
%run dataset.ipynb 
%run neuralnet.ipynb
%run training.ipynb

# if __name__ == '__main__':

In [12]:
    parser = argparse.ArgumentParser(usage='A training program for classifying the EMNIST dataset')
    parser.add_argument('-f', '--file', type=str, help='Path .mat file data', required=True)
    parser.add_argument('--width', type=int, default=28, help='Width of the images')
    parser.add_argument('--height', type=int, default=28, help='Height of the images')
    parser.add_argument('--max', type=int, default=None, help='Max amount of data to use')
    parser.add_argument('--epochs', type=int, default=18, help='Number of epochs to train on')
    parser.add_argument('--verbose', action='store_true', default=False, help='Enables verbose printing')

    #     args = parser.parse_args()

    data_file = 'emnist-byclass.mat'
    data_path = os.path.dirname(os.path.realpath(sys.argv[1])) + '/data/' + data_file
#     args = parser.parse_args(['-f', data_path, '--max', 10000])
    args = parser.parse_args(['-f', data_path])

    bin_dir = os.path.dirname(os.path.realpath(sys.argv[1])) + '/bin'
    if not os.path.exists(bin_dir):
        os.makedirs(bin_dir)    

In [8]:
    training_data = load_data(args.file, width=args.width, height=args.height, max_data=args.max, verbose=args.verbose)

In [13]:
    model = build_net(training_data, width=args.width, height=args.height, verbose=args.verbose)

In [ ]:
    train(model, training_data, epochs=args.epochs)

Train on 697932 samples, validate on 116323 samples
Epoch 1/18
 29952/697932 [>.............................] - ETA: 3260s - loss: 2.0149 - acc: 0.4928